In [15]:
%run C:/Users/admin/Desktop/EeD/Python/Project/Preprocessing.ipynb

[8.57524322e-03 1.88883990e-02 1.93219955e-02 1.38695115e-02
 9.83899967e-03 1.64104661e-02 1.60283129e-02 1.37872793e-02
 1.67041053e-02 4.03335108e-03 1.97271942e-02 1.69880217e-02
 1.83832052e-02 1.10206581e-02 1.77891041e-03 1.92224861e-02
 9.10963609e-03 3.13436054e-03 5.38128330e-03 7.81530310e-03
 1.01519884e-02 2.28094880e-03 1.54349838e-02 1.01977751e-02
 1.59063825e-02 8.02494093e-03 1.75721089e-02 1.66537704e-02
 1.52448504e-02 5.88378093e-03 2.64393807e-03 4.59109757e-03
 2.02759161e-04 3.17726372e-03 2.18799581e-02 1.86212060e-02
 3.24253463e-04 1.61766479e-03 1.79285512e-03 3.98049550e-03
 3.13249573e-03 5.42361661e-04 6.46529698e-04 1.46510394e-03
 1.98678510e-03 9.07283598e-03 2.42952742e-03 1.10417141e-03
 9.19444359e-03 2.56784509e-03 2.10261925e-03 1.82369197e-03
 2.99356895e-03 1.71753372e-04 1.18824370e-04 7.45469276e-03
 3.54947231e-03 3.17391733e-03 4.02264256e-04 9.68415727e-03
 2.26078459e-03 2.28971807e-03 8.84613372e-04 7.24426990e-04
 1.35564817e-04 5.060955

In [21]:
ah_data_target = ah_data['SalePrice']
ah_data_sf = pd.get_dummies(ah_data_sf)

In [57]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import ElasticNet
lm = linear_model.LinearRegression()

#split train and test data
X_train, X_test, y_train, y_test = train_test_split(ah_data_sf, ah_data_target, test_size = 0.6, random_state = 10) 


#fit the multiple linear regression model
lm.fit(X_train, y_train)

#make predictions on the testing set
y_train_pred = lm.predict(X_train)
y_test_pred = lm.predict(X_test)
print("Train MSE: ", metrics.mean_absolute_error(y_train, y_train_pred))
print("Test MSE: ", metrics.mean_absolute_error(y_test, y_test_pred))
print("Test RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))


Train MSE:  28959.960899042107
Test MSE:  31834.244640582892
Test RMSE:  45973.928931541974


In [58]:
#fitting Ridge Regression Model
lr = linear_model.Ridge(alpha = 0.001)
lr.fit(X_train, y_train)

#make predictions and evaluate
y_train_pred2 = lr.predict(X_train)
y_test_pred2 = lr.predict(X_test)
print("Train MSE: ", metrics.mean_absolute_error(y_train, y_train_pred2))
print("Test MSE: ", metrics.mean_absolute_error(y_test, y_test_pred2))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2)))


Train MSE:  28959.96046035168
Test MSE:  31834.23381351325
Train RMSE:  45973.91784899881


In [59]:
#fitting the LASSO Regression Model
ls = linear_model.Lasso(alpha = 0.001)
ls.fit(X_train, y_train)
#make predictions and evaluate
y_train_pred3 = ls.predict(X_train)
y_test_pred3 = ls.predict(X_test)
print("Train MSE: ", metrics.mean_absolute_error(y_train, y_train_pred3))
print("Test MSE: ", metrics.mean_absolute_error(y_test, y_test_pred3))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred3)))


Train MSE:  28959.958374835798
Test MSE:  31834.240629523927
Train RMSE:  45973.92424980893


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [61]:
en = linear_model.ElasticNet(alpha = 0.1, l1_ratio = 0.5)
en.fit(X_train, y_train)
#make predictions and evaluate
y_train_pred4 = en.predict(X_train)
y_test_pred4 = en.predict(X_test)
print("Train MSE: ", metrics.mean_absolute_error(y_train, y_train_pred4))
print("Test MSE: ", metrics.mean_absolute_error(y_test, y_test_pred4))
print("Train RMSE: ", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred4)))


Train MSE:  28985.098630361277
Test MSE:  31679.899582975762
Train RMSE:  45858.55369334189


C:\Users\admin\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [49]:
mae_train = [metrics.mean_absolute_error(y_train, y_train_pred), metrics.mean_absolute_error(y_train, y_train_pred2), metrics.mean_absolute_error(y_train, y_train_pred3)]

mae_test = [metrics.mean_absolute_error(y_test, y_test_pred),metrics.mean_absolute_error(y_test, y_test_pred2),metrics.mean_absolute_error(y_test, y_test_pred3)]

rmse_test = [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2)), np.sqrt(metrics.mean_squared_error(y_test, y_test_pred3))]

df = {"MAE_train": mae_train, "MAE_test": mae_test, "RMSE_test":rmse_test}

evaluation_table = pd.DataFrame(data = df)
mapper = {0:"Linear", 1:"Ridge", 2:"Lasso"}
evaluation_table.rename(mapper = mapper, axis='index', copy=True, inplace=True)
evaluation_table

,MAE_train,MAE_test,RMSE_test
Linear,30164.467470,30534.610884,45650.112138
Ridge,30164.463252,30534.612164,45650.112158
Lasso,30164.467144,30534.608965,45650.109643
